# BEFANA: A Tool for Biodiversity-Ecosystem Functioning Assessment by Network Analysis

## Part 1: Preparing the environment and loading the data

In [ ]:
!pip -q install "jupyterlab==3.2.*"
!pip -q install "ipywidgets==7.6.*"
!pip -q install "matplotlib==3.5.*"
!pip -q install "pandas==1.4.*"
!pip -q install "networkx==2.6.*"

In [3]:
import io
import codecs
import pandas as pd

import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual,  HBox, VBox
from IPython.display import display, HTML

from utils.ui_functions import StopExecution, make_download_button, alert_user
from utils.conf import my_style, my_layout

### 1.1 Loading the network matrix

Load the ajacency matrix containing the information on the interaction network and proceed to the next cell. The sample data can be downloaded [here.](https://raw.githubusercontent.com/MartinMarzi/BEFANA/main/data/sfw_matrix.csv)

In [4]:
widgets.FileUpload(accept='.csv',multiple=False)
matrix_uploader = widgets.FileUpload()
display(matrix_uploader)

FileUpload(value={}, description='Upload')

After the below cell is run the loaded adjacnecy matrix is displayed and one can use the GUI sliders to adjust its view.

In [6]:
try:
    matrix_file_name, matrix_file_data = list(matrix_uploader.value.items())[0]
except Exception:
    alert_user('Please load the food web matrix data file the cell above!\nThe program cannot continue without this data.')
    raise StopExecution    
sfw_matrix = pd.read_csv(io.StringIO(codecs.decode(matrix_file_data['content'], encoding="utf-8")),
                         header=0, 
                         index_col=0, 
                         sep=",")

if sfw_matrix.shape[0]!=sfw_matrix.shape[1]:
    alert_user('Invalid graph matrix: it must have NxN shape.')
    raise StopExecution

@interact(nrows=widgets.IntSlider(description='Rows:', min=3, max=sfw_matrix.shape[1], step=1, value=5),
          ncols=widgets.IntSlider(description='Columns:', min=3, max=sfw_matrix.shape[0], step=1, value=10))
def show_soil_food_web_matrix(nrows, ncols):
    pd.set_option("display.max_rows", nrows)
    pd.set_option("display.max_columns", ncols)
    display(sfw_matrix)

interactive(children=(IntSlider(value=5, description='Rows:', max=24, min=3), IntSlider(value=10, description=…

### 1.2 Loading the trophic level matrix

If we have the trophic level data it is possible to load it here. It must have the `Nx1` shape. If no such data is available, the following two cells are skipped and the trophic level will not be used (all nodes will have the same value). The sample data can be downloaded [here.](https://raw.githubusercontent.com/MartinMarzi/BEFANA/main/data/sfw_trophic_levels.csv) 

In [7]:
widgets.FileUpload(accept='csv',multiple=False)
trophic_level_uploader = widgets.FileUpload()
display(trophic_level_uploader)

FileUpload(value={}, description='Upload')

In [9]:
try:
    trophic_level_file_name, trophic_level_file_data = list(trophic_level_uploader.value.items())[0]
except Exception:
    alert_user('Please load the file with trophic level data in the cell above!\nThe program cannot continue without trophic level data.')
    raise StopExecution
trophic_levels = pd.read_csv(io.StringIO(codecs.decode(trophic_level_file_data['content'], encoding='utf-8')),
                             index_col=0, header=None, delimiter=",", names=['trophic_level'])

if trophic_levels.shape[1]!=1:
    alert_user('Invalid thropic matrix: it must have Nx1 shape.')
    raise StopExecution
if trophic_levels.shape[0] != sfw_matrix.shape[0]:
    alert_user('Invalid number of rows in the trophic matrix: trophic level must be defined for all organisms (nodes).')
    raise StopExecution
    
pd.set_option('display.max_rows', None)
display(trophic_levels.sort_values('trophic_level'))
trophic_levels = trophic_levels.squeeze()

,trophic_level
49 Primary (heterotrophic) producer fungi,0
48 Primary (heterotrophic) producer bacteria,0
#0 Plant autotrophs (roots),0
11 Plant-feeding nematode,1
22 Microphytophage mite (feeding on fungi),1
"41 ""Passive lifestage, substrate-related nematode""",1
37 Bacterivore flagellates,1
36 Bacterivore amoebae,1
12 Macrophytophage and panphytophage mite,1
34 Bacterivore enchytraeid,1


Finally, we store the created data structures so they will be available in other notebooks.

In [10]:
sfw_matrix.to_pickle('results/sfw_matrix.pickle')
trophic_levels.to_pickle('results/trophic_levels.pickle')